In [20]:
import jellyfish
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import dateutil.parser as parser
from geopy.geocoders import Nominatim
import pycountry
import time
from datetime import date
from dateutil.relativedelta import relativedelta
nlp = en_core_web_sm.load()
nltk.download('vader_lexicon')

df = pd.read_excel("NUS sample names_V2.xlsx", engine="openpyxl")
df = df.where(pd.notnull(df), None)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dylan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Web Scraper

In [140]:
# Main Function
def search_articles_on_individual(individual_dict, no_of_articles=10):
    def generate_link(person_dict, attributes_used = ['name', 'alias'], keywords=['crimes', 'sentenced']):
        link_start = "https://www.google.com/search?q="
        link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
        link_query = ""

        for attributes in attributes_used:
            temp_attr = person_dict[attributes]
            if temp_attr is not None:
                temp_attr = str(temp_attr)
                link_query += temp_attr.replace(' ', '+') + '+'       
                
        links = []
        for keyword in keywords:
            temp_search_link = link_start + link_query + keyword + link_end + "&num=" + str(no_of_articles)
            links.append(temp_search_link)
        return links
    
    def article_extraction(link):
        article = Article(link)
        article.download()
        try:
            article.parse()
        except:
            pass
        return article.text

    def parse(text):
        #try:     
        doc = nlp(text)
        tags = [[X.text, X.label_] for X in doc.ents]
        labels = [x.label_ for x in doc.ents]
        items = [x.text for x in doc.ents]

        return tags

    def find_names(tags):
        names = []
        for tag in tags:
            if tag[1] == 'PERSON':
                names.append(tag[0])
        return names

    def sentiment_analysis(text):
        return vader.polarity_scores(text)
    
    def time_to_months(time):
        if 'weeks' in time:
            return 0
        else:
            return int(time.split(' month')[0])

    search_links = generate_link(individual_dict)
    
    unique_links_checker = []
    
    output = []
    for x in search_links:
        print("SEARCHING THIS", x)
        req = Request(x, headers = {'User-Agent': 'Mozilla/5.0'})

        webpage = urlopen(req).read()

        with requests.Session() as c:
            soup = BeautifulSoup(webpage, 'html5lib')
            #print(soup)
            for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
                current_dict = {}
                raw_link = (item.find('a', href = True)['href'])
                link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
                if link not in unique_links_checker:
                    unique_links_checker.append(link)
                    title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).get_text()
                    description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
                    time = description.split(" · ")[0]
                    #print(description)
                    descript = description.split(" · ")[1]
                    current_dict['title'] = title
                    current_dict['time'] = time
                    try:
                        current_dict['year_of_birth'] = (date.today() - relativedelta(months = time_to_months(time))).year - individual_dict['year_of_birth']
                    except TypeError as e:
                        current_dict['year_of_birth'] = 0
                    current_dict['description'] = descript
                    current_dict['link'] = link
                    current_dict['text'] = article_extraction(link)
                    current_dict['sentiment?'] = sentiment_analysis(article_extraction(link))
                    
                    
                    parsed_description = parse(description)
                    names_in_description = find_names(parsed_description)
                    
                    parsed_text = parse(article_extraction(link))
                    names_in_text = find_names(parsed_text)

                    names_list = Counter(names_in_description + names_in_text)
                    current_dict['names_list'] = names_list
                    
                    output.append(current_dict)
                else:
                    pass
    return output

new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'bankrupt': -100,
    'fraud': -100,
    'crime': -100,
    'prison': -100,
    'embezzlement': -100,
    'money_trail':-100,
    'jail': -100,
    'sentenced':-100,
    'forfeiture': -100,
    'ill-gotten gain':-100,
    'laundering':-100
}

vader = SentimentIntensityAnalyzer()

vader.lexicon.update(new_words)

In [141]:
def preprocess_df_to_dict(df):
    def get_year(date):
        try:
            parser_obj = parser.parse(str(date))
            return parser_obj.year
        except:
            return None

    def get_month(date):
        if len(str(date))>4:
            try:
                return parser.parse(str(date)).month
            except:
                return None
        else:
            return None
            
    def get_day(date):
        if len(str(date))>4:
            try:
                return parser.parse(str(date)).day
            except:
                return None
        else:
            return None    
    
    df_dict_list = df.to_dict('records')
    cleaned_dict_list = []
    for record in df_dict_list:   
        current_record = {
            'name': record['Name to be screened'],
            'alias' : record['Alias name'],
            'year_of_birth': get_year(record['Date of birth']),
            'month_of_birth': get_month(record['Date of birth']),
            'day_of_birth': get_day(record['Date of birth']),
            'gender': record['Gender'],
            'nationality': record['Nationality'],
            ### delete these later on, for testing only###
            'type_of_error': record['Type of variation (if any)'],
            'actual_name': record['Actual name'],
        }
        cleaned_dict_list.append(current_record)
    return cleaned_dict_list

In [142]:
df_dict = preprocess_df_to_dict(df)
test_record_1 = df_dict[0]
test_record_18 = df_dict[15]
print(test_record_18)

{'name': 'Lange Vivian', 'alias': None, 'year_of_birth': 1997, 'month_of_birth': None, 'day_of_birth': None, 'gender': 'Female', 'nationality': 'Singapore', 'type_of_error': '-', 'actual_name': 'Lange Vivian'}


In [143]:
test_query = search_articles_on_individual(test_record_18, 10)
pd.DataFrame(test_query)

SEARCHING THIS https://www.google.com/search?q=Lange+Vivian+crimes&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2&num=10
SEARCHING THIS https://www.google.com/search?q=Lange+Vivian+sentenced&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2&num=10


,title,time,year_of_birth,description,link,text,sentiment?,names_list
0,Woman first to be charged under new anti-money...,14 months ago,23,Lange Vivian was on Wednesday (June 24) charge...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - A 23-year-old woman is the first p...,"{'neg': 0.439, 'neu': 0.553, 'pos': 0.008, 'co...","{'Lange Vivian': 1, 'Lange': 3, 'Toa Payoh and..."
1,American Horror Stories Episode 7 Explained by...,1 month ago,24,I would go upstairs and hang out in Vivian and...,https://collider.com/american-horror-stories-e...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Vivian': 1, 'Ben': 1}"
2,[PHOTOS] American Horror Story's Best Characte...,1 month ago,24,(Spoiler alert: All of Lange's can be found he...,https://tvline.com/lists/american-horror-story...,"Ready to revisit 10 years’ worth of murder, ma...","{'neg': 0.095, 'neu': 0.758, 'pos': 0.147, 'co...","{'Lange': 2, 'Kit': 1, 'Ryan Murphy': 1, 'Brad..."
3,Singapore Cracks Down on Unlicensed Crypto Sel...,14 months ago,23,"... of crime from victims of online scams, the...",https://www.finews.asia/finance/32107-singapor...,Authorities in the city-state have charged a 2...,"{'neg': 0.589, 'neu': 0.382, 'pos': 0.029, 'co...",{'Vivian Lange': 2}
4,Corey Taylor Reveals What He Thinks Is 'The Pe...,3 weeks ago,24,"You listen to it and you're like, 'This is cri...",https://loudwire.com/corey-taylor-reveals-perf...,The buzz this year has been all about the 30th...,"{'neg': 0.073, 'neu': 0.751, 'pos': 0.176, 'co...","{'Mutt Lange': 1, 'Metallica': 4, 'Corey Taylo..."
5,A long history of bigotry against Asian Americans,5 months ago,24,They have not yet charged him with hate crimes...,https://news.harvard.edu/gazette/story/2021/03...,"Last March, Vivian Shaw, a College Fellow in t...","{'neg': 0.102, 'neu': 0.832, 'pos': 0.065, 'co...","{'Vivian Shaw': 3, 'Jason Beckfield': 1, 'Shaw..."
6,How Vivian Gornick Reinvigorated Political Wri...,16 months ago,23,Vivian Gornick specializes in personal narrati...,https://newrepublic.com/article/157527/vivian-...,Gornick’s interviewees describe how communism ...,"{'neg': 0.066, 'neu': 0.767, 'pos': 0.167, 'co...","{'Vivian Gornick': 1, 'Lange': 1, 'Gornick': 5..."
7,Healthcare disparities: The inside story from ...,1 month ago,24,News,https://www.postbulletin.com/rochester-in-colo...,"Accustomed to asking the questions, NewsMD hea...","{'neg': 0.042, 'neu': 0.869, 'pos': 0.09, 'com...","{'Vivian Williams': 1, 'Andre Crockett': 1, 'S..."
8,Durham filmmaker to direct new HBO Max documen...,6 months ago,24,Hill is best known for directing Kinston chef ...,https://www.newsobserver.com/news/local/articl...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Hill': 1, 'Vivian Howard': 1}"
9,MIGALHAS nº 5.174,3 weeks ago,24,"Para o ministro, não existe o chamado ""crime d...",https://www.migalhas.com.br/amanhecidas/350730...,"Quinta-Feira, 26 de agosto de 2021 - Migalhas ...","{'neg': 0.068, 'neu': 0.928, 'pos': 0.005, 'co...","{'Para': 3, 'Tese de mestrado': 1, 'Vivian Lui..."


# Name Matching

In [8]:
def ER_name_matching(name1, name2):
    def split_name_list(name):
        name = name.lower()
        output = name.split(" ")
        return output

    def preprocess_name(names_dict, word):
        for key, value in names_dict.items():
            if word in value:
                return key
        else:
            return word

    def stitch_name(list1):
        output = ''
        for x in range(len(list1)):
            if x==0:
                output += list1[x]
            else:
                output += ' ' + list1[x]
        return output

    def phonetic_comparison(list1, list2):
        meta_list1 = []
        meta_list2 = []
        nysiis_list1 = []
        nysiis_list2 = []
        for name_1 in list1:
            meta_list1.append(jellyfish.metaphone(name_1))
            nysiis_list1.append(jellyfish.nysiis(name_1))
        for name_2 in list2:
            meta_list2.append(jellyfish.metaphone(name_2))
            nysiis_list2.append(jellyfish.nysiis(name_2))
        if (set(meta_list1) == set(meta_list2)) or (set(nysiis_list1) == set(nysiis_list2)):
            return True
        else:
            return False
    
    def excel_to_dict(excel_file):
        excel_df = pd.read_excel(excel_file)
        excel_df.value.apply(str)
        before_transformation = dict(zip(excel_df.key, excel_df.value))
        dictionary = {key: [val for val in value.split(',')] for key, value in before_transformation.items()}
        return dictionary
            
    names_dict = excel_to_dict('names_dict.xlsx') 
    
    # START #
    ### Change this if needed ###
    threshold = 89
    #############################
    
    split_list_1 = split_name_list(name1)
    split_list_2 = split_name_list(name2) 
 
    
    for i in range(len(split_list_1)):
        split_list_1[i] = preprocess_name(names_dict, split_list_1[i])        
    for i in range(len(split_list_2)):
        split_list_2[i] = preprocess_name(names_dict, split_list_2[i])
    
    stitched_name1 = stitch_name(split_list_1)
    stitched_name2 = stitch_name(split_list_2)
    
    # 1st layer of testing: Token Sort Ratio with threshold
    score1 = fuzz.token_sort_ratio(stitched_name1, stitched_name2)
    if score1 >= threshold:
        # score_list.append(score1)
        return score1
        # do something
# 4) 2nd layer of testing - Metaphone and NYSIIS phonetic encoding - DONE
    else: 
        matched_phonetic = phonetic_comparison(split_list_1, split_list_2)
        if matched_phonetic:
            return threshold # assumption that phonetic match will give threshold score
        else: 
            return None
    
    try:
        return score1
    except:
        pass

# Nationality Matching

In [9]:
# hlpr func: get country by cities, states name
def get_country(gpe):
    geolocator = Nominatim(user_agent = "geoapiExercises")
    location = geolocator.geocode(gpe)
    if location:
        loc_lst = location.address.split(',')
        return loc_lst[-1]
    return None

# hlpr func: return a list of countries names
def countries():
    return list(map(lambda x: x.name, list(pycountry.countries)))

# hlpr func: return True if name countains country name
def contain_country(word, ctry_lst):
    for ctry in ctry_lst:
        if ctry.lower() in word.lower():
            return True
    return False

# hlpr func: extract entities with tag 'GPE', 'ORG', 'NORP'
def search_target_ent(tags):
    country_lst = countries()
    tag_lst = []
    for i in range(len(tags)):
        if tags[i][1] == 'GPE' or tags[i][1] == 'ORG' or tags[i][1] == 'NORP':
            if contain_country(tags[i][0], country_lst):
                tag_lst.append(tags[i])
    return tag_lst

# hlpr func: return the odd of the person's nationality in the article is nat
def calc_odd_nationality(nat,lst):
    try:
        result = []
        for tag in lst:
            if nat.lower() in tag[0].lower():
                result.append(tag)
                continue
            if tag[1] == 'GPE' and get_country(tag[0]).lower() == nat.lower():
                result.append(tag)
        prob = 1 if ((len(lst) - len(result)) == 0 and len(result) > 0) else (len(result) / (len(lst) - len(result)))
        prob = 1 if prob > 1 else prob
        return prob
    except TypeError as e:
        pass

# hlpr func: return True if name fuzzy matching score > 80
def is_target(name, article_name):
    return fuzz.partial_ratio(name, article_name) > 80

# the main function for nationality matching
# return odd if target tags found else return N/A
def nationality_matching(tags, nationality, person):
    result = []
    try:
        for i in range(len(tags)):
            #if second item is a name
            if tags[i][1] == 'PERSON':
                
                # check if is target
                if is_target(person, tags[i][0]):
                    search = search_target_ent(tags)
                
                    if len(search) != 0:
                        return calc_odd_nationality(nationality, search)
        return 'N/A'
    except IndexError as e:
        pass

# hlpr func: parse text to tags
def parse(text):
        #try:     
        doc = nlp(text)
        tags = [[X.text, X.label_] for X in doc.ents]
        labels = [x.label_ for x in doc.ents]
        items = [x.text for x in doc.ents]

        return tags

# Gender Matching

In [10]:
# hlpr func: return True if token is a name and subject
def is_name_subj(token):
    return token.dep_ =='nsubj' and token.pos_ == 'PROPN'

def is_part_of_name(token):
    return (token.dep_ =='nsubj' or token.dep_ =='compound') \
        and token.pos_ == 'PROPN'

# hlpr func: return True if the token is a determiner: his, her, hers
def is_det(token):
    return token.pos_ == 'DET' and (token.dep_ == 'poss' or token.dep_ == 'attr')

# hlpr func: return True if the token is a pronoun: he, she, herself, himself
def is_pron(token):
    return token.pos_ == 'PRON' and \
        (token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' or token.dep_ == 'pobj' or token.dep_ == 'dobj')

# hlpr func: return the probability of the gender in article to the true gender
def calc_prob_gender(pron_lst, gender):
    male_pron = ['he', 'his', 'himself', 'him']
    female_pron = ['she', 'her', 'herself', 'hers']
    n_target = 0
    gdr_pron = []
    
    if gender.lower() == 'male':
        gdr_pron = male_pron
    else:
        gdr_pron = female_pron
        
    for pron in pron_lst:
        if pron in gdr_pron:
            n_target += 1
    return n_target / len(pron_lst) if len(pron_lst) else 0

# the main function in gender matching
def gender_matching(text, gender, name):
    try:
        pron_lst = ['he', 'his', 'himself', 'him', 'she', 'her', 'herself', 'hers']
        name_str = ''
        target_name = name.replace(" ", "")
        target_found = False
        res_lst = []
        
        # text tagging
        doc = nlp(text)
        
        i = 0
        while i < len(doc):
            # search for target name of subject form
            if is_name_subj(doc[i]):
                end_name = i
                start_name = i
                while is_part_of_name(doc[start_name]):
                    start_name -= 1
                start_name += 1
                while start_name <= end_name:
                    name_str += doc[start_name].text
                    start_name += 1
                i += 1
            if is_target(name_str, target_name):
                target_found = True
            else:
                name_str = ''
                target_found = False
            
            # if target name is found, search for pronouns, break if another name is found
            while target_found:
                if is_name_subj(doc[i]):
                    target_found = False
                    name_str = ''
                    break
                if is_det(doc[i]) or is_pron(doc[i]):
                    if (doc[i].text).lower() in pron_lst:
                        res_lst.append((doc[i].text).lower())
                        break
                i += 1
            if is_name_subj(doc[i]):
                continue
            i += 1    
    except IndexError as e:
        pass
    return calc_prob_gender(res_lst, gender)

## Age Matching

In [173]:
useless_dates = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','yesterday','today']

#index is index of person
def forward_searcher(index,tags):
    for i in range(index,len(tags)):
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
    return [None,None]

def backward_searcher(index,tags):
    i = index
    while i >= 0:
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
        else:
            i -=1

def detect_age(age,lst):
    try:
        if lst[1] is not None and lst[2] is not None:
            date1 = lst[1][0]
            date2 = lst[2][0]
            if (str(age) in date1) or (str(age) in date2):
                return True
        else:

            if lst[1] == None:
                if str(age) in lst[2][0]:
                    return True

            if lst[2] == None:
                if str(age) in lst[1][0]:
                    return True
    except TypeError as e:
        pass
    
    
def confirm_age(lst,age,threshold):
    iterating_lst = []
    plus = 1
    minus = -1
    for i in range(threshold):
        iterating_lst.append(age+plus)
        plus += 1
    for i in range(threshold):
        iterating_lst.append(age+minus)
        minus -=1 
    iterating_lst.append(age)
    
    for j in iterating_lst:
        if str(j) in lst[1][0]:
            return 1
    return 0

def age_matching(name_dict,tags,age):
    '''
    tags: parse(text)
    age: desired age to check
    '''
    for tag in tags:
        #if tag[1] == 'DATE':
            #print(tag)
        if str(age) in tag[0] or str(age+1) in tag[0] or str(age-1) in tag[0]:
            return 1
    result = []
    try:
        for i in range(len(tags)):
            #if second item is a name

            
            if tags[i][1] == 'PERSON':
                if tags[i][0] in name_dict:

                    forward_age = forward_searcher(i,tags)
                    backwards_age = backward_searcher(i,tags)
                    new_list = [tags[i],forward_age,backwards_age]
                    #new_list = [tags[i-1],tags[i],tags[i+1]]
                    #print(new_list)

                    if detect_age(age,new_list) and tags[i][0] in name_list:

                        #print(new_list)
                        #result += new_list

                        if str(age) in new_list[1][0]:
                            #print('****************')
                            #print([tags[i], new_list[1]])
                            return(confirm_age([tags[i],new_list[1]],age,3))


                        elif str(age) in new_list[2][0]:
                            #print('****************')
                            #print([tags[i],new_list[2]])
                            return(confirm_age([tags[i],new_list[2]],age,3))
                        
        return 0
    except IndexError as e:
        pass 
                

In [174]:
def entity_recognition_scoring(input_info, list_of_article_dicts):
    output = []
    input_name = input_info['name']

    for article in list_of_article_dicts:
        article_names_list = article['names_list'].most_common() 
        matched = False

        for each_name, each_count in article_names_list: ## as of now checking all names within the article, should we limit to e.g. top 3/5?
            if len(each_name.split()) == 1 and each_name in input_name:
                score = 100 ## if surname matches, default match score 100 
            else: 
                try: 
                    score = ER_name_matching(input_name, each_name)
                except ValueError as e:
                    pass
            if score is not None:
                matched = True
            if matched:
                break
     
        if matched:
            # keep article
            article['name_matching_score'] = score
            output.append(article)

            # nationality matching score
            article ['nationality_matching_score'] = nationality_matching(parse(article['text']), input_info['nationality'], input_info['name'])
            output.append(article)

            # gender matching score
            article ['gender_matching_score'] =gender_matching(article['text'], input_info['gender'], input_info['name'])
            output.append(article)
            
            article['age_matching_score'] = age_matching(article['names_list'],parse(article['text']),article['year_of_birth'])
            #print(article['year_of_birth'])
    
    return output
            
        
    ##### ADD IN SECONCARY IDENTIFIERS #######
    
    ##### ADD IN SCORING SYSTEM HERE #######    

In [175]:
test_record_18

{'name': 'Lange Vivian',
 'alias': None,
 'year_of_birth': 1997,
 'month_of_birth': None,
 'day_of_birth': None,
 'gender': 'Female',
 'nationality': 'Singapore',
 'type_of_error': '-',
 'actual_name': 'Lange Vivian'}

In [176]:
name_matched = entity_recognition_scoring(test_record_18, test_query)
pd.DataFrame(name_matched)

,title,time,year_of_birth,description,link,text,sentiment?,names_list,name_matching_score,nationality_matching_score,gender_matching_score,age_matching_score
0,Woman first to be charged under new anti-money...,14 months ago,23,Lange Vivian was on Wednesday (June 24) charge...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - A 23-year-old woman is the first p...,"{'neg': 0.439, 'neu': 0.553, 'pos': 0.008, 'co...","{'Lange Vivian': 1, 'Lange': 3, 'Toa Payoh and...",100,1,0,1
1,Woman first to be charged under new anti-money...,14 months ago,23,Lange Vivian was on Wednesday (June 24) charge...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - A 23-year-old woman is the first p...,"{'neg': 0.439, 'neu': 0.553, 'pos': 0.008, 'co...","{'Lange Vivian': 1, 'Lange': 3, 'Toa Payoh and...",100,1,0,1
2,Woman first to be charged under new anti-money...,14 months ago,23,Lange Vivian was on Wednesday (June 24) charge...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - A 23-year-old woman is the first p...,"{'neg': 0.439, 'neu': 0.553, 'pos': 0.008, 'co...","{'Lange Vivian': 1, 'Lange': 3, 'Toa Payoh and...",100,1,0,1
3,American Horror Stories Episode 7 Explained by...,1 month ago,24,I would go upstairs and hang out in Vivian and...,https://collider.com/american-horror-stories-e...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Vivian': 1, 'Ben': 1}",100,N/A,0,0
4,American Horror Stories Episode 7 Explained by...,1 month ago,24,I would go upstairs and hang out in Vivian and...,https://collider.com/american-horror-stories-e...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Vivian': 1, 'Ben': 1}",100,N/A,0,0
5,American Horror Stories Episode 7 Explained by...,1 month ago,24,I would go upstairs and hang out in Vivian and...,https://collider.com/american-horror-stories-e...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Vivian': 1, 'Ben': 1}",100,N/A,0,0
6,[PHOTOS] American Horror Story's Best Characte...,1 month ago,24,(Spoiler alert: All of Lange's can be found he...,https://tvline.com/lists/american-horror-story...,"Ready to revisit 10 years’ worth of murder, ma...","{'neg': 0.095, 'neu': 0.758, 'pos': 0.147, 'co...","{'Lange': 2, 'Kit': 1, 'Ryan Murphy': 1, 'Brad...",100,N/A,0,1
7,[PHOTOS] American Horror Story's Best Characte...,1 month ago,24,(Spoiler alert: All of Lange's can be found he...,https://tvline.com/lists/american-horror-story...,"Ready to revisit 10 years’ worth of murder, ma...","{'neg': 0.095, 'neu': 0.758, 'pos': 0.147, 'co...","{'Lange': 2, 'Kit': 1, 'Ryan Murphy': 1, 'Brad...",100,N/A,0,1
8,[PHOTOS] American Horror Story's Best Characte...,1 month ago,24,(Spoiler alert: All of Lange's can be found he...,https://tvline.com/lists/american-horror-story...,"Ready to revisit 10 years’ worth of murder, ma...","{'neg': 0.095, 'neu': 0.758, 'pos': 0.147, 'co...","{'Lange': 2, 'Kit': 1, 'Ryan Murphy': 1, 'Brad...",100,N/A,0,1
9,Singapore Cracks Down on Unlicensed Crypto Sel...,14 months ago,23,"... of crime from victims of online scams, the...",https://www.finews.asia/finance/32107-singapor...,Authorities in the city-state have charged a 2...,"{'neg': 0.589, 'neu': 0.382, 'pos': 0.029, 'co...",{'Vivian Lange': 2},100,N/A,0,1


In [168]:
name_matched

[{'title': 'Woman first to be charged under new anti-money laundering law',
  'time': '14 months ago',
  'year_of_birth': 23,
  'description': 'Lange Vivian was on Wednesday (June 24) charged with one count of ... of crime from victims of online scams," a police spokesman said.',
  'link': 'https://www.straitstimes.com/singapore/courts-crime/woman-first-to-be-charged-under-new-anti-money-laundering-law',
  'text': 'SINGAPORE - A 23-year-old woman is the first person to be charged under a new law that fights money laundering and terrorism financing.\n\nLange Vivian was on Wednesday (June 24) charged with one count of providing payment services without a licence under the Payment Services Act.\n\nThe law, which also aims to strengthen consumer protection in the use of e-payments, was introduced on Jan 28.\n\nThe Singaporean allegedly provided a digital payment token service between Feb 27 and 28 this year.\n\nIn a statement on Tuesday, the police said that she allegedly received at least